## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-04-23-57-26 +0000,nypost,Manhattan retail lease rebound continues — but...,https://nypost.com/2026/02/04/business/manhatt...
1,2026-02-04-23-54-24 +0000,nypost,VP Vance to lead California anti-fraud task fo...,https://nypost.com/2026/02/04/us-news/vp-vance...
2,2026-02-04-23-54-00 +0000,wsj,Arm Holdings Expects Slower Revenue Growth in ...,https://www.wsj.com/business/earnings/arm-hold...
3,2026-02-04-23-48-00 +0000,wsj,Fed’s Cook Focused on Inflation Risks as Great...,https://www.wsj.com/economy/central-banking/fe...
4,2026-02-04-23-47-33 +0000,nypost,Owners of buzzy LA restaurant used millions in...,https://nypost.com/2026/02/04/us-news/owners-o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2640/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
73,trump,40
65,epstein,26
246,new,25
12,california,14
93,immigration,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
385,2026-02-04-00-00-00 +0000,wsj,Stephen Miller called Alex Pretti a “domestic ...,https://www.wsj.com/politics/policy/ice-stephe...,107
372,2026-02-04-01-07-00 +0000,wsj,Fed governor Stephen Miran said he resigned fr...,https://www.wsj.com/economy/central-banking/st...,98
19,2026-02-04-23-11-15 +0000,nypost,Trump gets help in his critical minerals race ...,https://nypost.com/2026/02/04/us-news/house-pa...,92
147,2026-02-04-19-57-31 +0000,nypost,President Trump called Savannah Guthrie to off...,https://nypost.com/2026/02/04/us-news/presiden...,90
217,2026-02-04-16-39-42 +0000,nypost,New Epstein emails about top Goldman lawyer Ka...,https://nypost.com/2026/02/04/business/epstein...,85


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
385,107,2026-02-04-00-00-00 +0000,wsj,Stephen Miller called Alex Pretti a “domestic ...,https://www.wsj.com/politics/policy/ice-stephe...
217,85,2026-02-04-16-39-42 +0000,nypost,New Epstein emails about top Goldman lawyer Ka...,https://nypost.com/2026/02/04/business/epstein...
372,66,2026-02-04-01-07-00 +0000,wsj,Fed governor Stephen Miran said he resigned fr...,https://www.wsj.com/economy/central-banking/st...
16,48,2026-02-04-23-20-43 +0000,nypost,Hochul’s anti-ICE bill will backfire and encou...,https://nypost.com/2026/02/04/us-news/nassau-c...
166,46,2026-02-04-19-10-44 +0000,latimes,"Supreme Court, with no dissents, rejects GOP c...",https://www.latimes.com/politics/story/2026-02...
184,45,2026-02-04-18-13-46 +0000,cbc,Washington Post staff laid off while in Milan ...,https://www.cbc.ca/sports/olympics/winter/mila...
379,44,2026-02-04-00-31-15 +0000,nypost,Netflix co-CEO grilled by senators as Warner B...,https://nypost.com/2026/02/03/business/netflix...
248,42,2026-02-04-14-52-00 +0000,wsj,"America’s private sector added 22,000 jobs las...",https://www.wsj.com/economy/jobs/adp-numbers-s...
245,41,2026-02-04-15-00-14 +0000,nyt,Minneapolis Police Chief Fears Long-Term Damag...,https://www.nytimes.com/2026/02/04/us/brian-oh...
222,40,2026-02-04-16-21-17 +0000,nypost,Tempers flare between Dem Rep. Maxine Waters a...,https://nypost.com/2026/02/04/us-news/tempers-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
